In [5]:
import pandas as pd
import openpyxl
import itertools
import io
from datetime import datetime
from pathlib import Path

In [6]:
def read_file(filepath):    
    def open_file_skip_lines(filepath, num_lines_to_skip=25):
        with open(filepath, 'r', encoding='gbk') as f:
            # 使用 islice 跳过前 num_lines_to_skip 行
            remaining_lines = itertools.islice(f, num_lines_to_skip, None)
            
            # 将剩下的行内容合并成一个字符串
            content_after_skip = "".join(remaining_lines)
            
            # 将字符串内容封装成一个 io.StringIO 对象，使其具有文件对象的行为
            return io.StringIO(content_after_skip)
    fields = ['time', 'type', 'counterparty', 'account', 'name', 'direction', 'amount', 'payment', 'status', '_1', '_2', '_3', '_4']
    file_io = open_file_skip_lines(filepath)
    raw = pd.read_csv(file_io, encoding='gbk', names=fields)

    return raw
    
    # return pd.concat([head, content], axis=0)
    # return content

In [7]:
raw = read_file('alipay_250813_250916.csv')
raw.head()

,time,type,counterparty,account,name,direction,amount,payment,status,_1,_2,_3,_4
0,2025-09-16 13:27:02,酒店旅游,绿城留庄杭州酒店管理有限公司,/,绿城留庄（杭州）酒店管理有限公司,支出,10.0,中国银行储蓄卡(7633),交易成功,2025091622001490961456555437\t,3120170922088721571202070132460000225091605373...,NaN,NaN
1,2025-09-16 08:19:58,餐饮美食,绿城留庄(杭州)酒店管理有限公,/,条码支付-杭研咖啡-10065942888012509160819560715,支出,1.5,中国银行储蓄卡(7633),交易成功,2025091622001490961455533214\t,6312660000489000009000033214825259\t,NaN,NaN
2,2025-09-15 19:47:03,交通出行,滴滴出行,chu***@didichuxing.com,滴滴快车打车-胡师傅-09月14日行程,支出,28.9,中国银行储蓄卡(7633),交易成功,2025091522001490961401308533\t,210_202509158437952001215291\t,NaN,NaN
3,2025-09-15 12:51:09,餐饮美食,饿了么,e50***@alibaba-inc.com,瑞幸咖啡(益展商务大厦店)外卖订单,支出,16.7,中国银行储蓄卡(7633),交易成功,2025091522001190961455437635\t,13120600725091584699442986026\t,NaN,NaN
4,2025-09-14 18:06:28,交通出行,杭州杭港地铁五号线有限公司,zfb***@mtrhz.com.cn,地铁-大运河-18:06:31-正常行程票款扣费,支出,4.0,中国银行储蓄卡(7633),交易成功,2025091422001490961448050179\t,051301162509141806281000000001443576797\t,NaN,NaN


In [8]:
def clean_raw(raw: pd.DataFrame, idx=False):
    raw = raw.copy()
    if idx:
        raw = raw.iloc[idx]

    # Clean each columns
    raw.drop(columns=['account', '_1', '_2', '_3', '_4'], inplace=True)
    raw.time = raw.time = raw.time.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    refund_idx = raw[(raw.status == '退款成功') | (raw.status == '交易关闭')].index
    raw.drop(index=refund_idx, inplace=True)
    raw.direction = raw.direction.map({'支出': 0, '不计收支': -1, '收入': 1})

    # Handle 'direction == -1' cases:
    case_idx = raw[raw.direction == -1].index
    print(case_idx)
    rec_to_concat = pd.DataFrame()
    for i in case_idx:
        if raw.loc[i, 'name'].startswith('余额宝-转出'):
            # Add a new record
            raw.loc[i, 'direction'] = 1
            new_rec = raw.loc[i].copy()
            new_rec['counterparty'] = '余额宝-转出到银行卡'
            new_rec['direction'] = 0
            new_rec['payment'] = '余额宝'
            rec_to_concat = pd.concat([rec_to_concat, new_rec.to_frame().T], ignore_index=True)
            
        elif raw.loc[i, 'name'].endswith('转入'):
            raw.loc[i, 'direction'] = 0
            raw.loc[i, 'counterparty'] = '银行卡-转入到余额宝'
            new_rec = raw.loc[i].copy()
            new_rec['counterparty'] = '余额宝-转入'
            new_rec['direction'] = 1
            new_rec['payment'] = '余额宝'
            rec_to_concat = pd.concat([rec_to_concat, new_rec.to_frame().T], ignore_index=True)

        elif raw.loc[i, 'name'].startswith('提现'):
            raw.loc[i, 'direction'] = 0
            new_rec = raw.loc[i].copy()
            new_rec['counterparty'] = '提现'
            new_rec['direction'] = 1
            new_rec['payment'] = '中国银行储蓄卡(7633)'
            rec_to_concat = pd.concat([rec_to_concat, new_rec.to_frame().T], ignore_index=True)

        elif raw.loc[i, 'name'].endswith('收益发放'):
            raw.loc[i, 'direction'] = 1
            raw.loc[i, 'counterparty'] = raw.loc[i, 'name']
    
    raw = pd.concat([raw, rec_to_concat], ignore_index=True)
            
    return raw

In [9]:
cleaned = clean_raw(raw)
cleaned

Index([42, 51, 68, 72, 77, 83, 85, 88, 93, 104, 108, 109, 110, 114, 118], dtype='int64')


,time,type,counterparty,name,direction,amount,payment,status
0,2025-09-16 13:27:02,酒店旅游,绿城留庄杭州酒店管理有限公司,绿城留庄（杭州）酒店管理有限公司,0,10.0,中国银行储蓄卡(7633),交易成功
1,2025-09-16 08:19:58,餐饮美食,绿城留庄(杭州)酒店管理有限公,条码支付-杭研咖啡-10065942888012509160819560715,0,1.5,中国银行储蓄卡(7633),交易成功
2,2025-09-15 19:47:03,交通出行,滴滴出行,滴滴快车打车-胡师傅-09月14日行程,0,28.9,中国银行储蓄卡(7633),交易成功
3,2025-09-15 12:51:09,餐饮美食,饿了么,瑞幸咖啡(益展商务大厦店)外卖订单,0,16.7,中国银行储蓄卡(7633),交易成功
4,2025-09-14 18:06:28,交通出行,杭州杭港地铁五号线有限公司,地铁-大运河-18:06:31-正常行程票款扣费,0,4.0,中国银行储蓄卡(7633),交易成功
...,...,...,...,...,...,...,...,...
113,2025-08-13 12:02:07,餐饮美食,美团,苞米熟了柴火铁锅炖-美团App-25081311100300001302941341770010,0,147.0,余额宝,交易成功
114,2025-08-13 06:01:23,投资理财,余额宝-2025.08.12-收益发放,余额宝-2025.08.12-收益发放,1,0.04,余额宝,交易成功
115,2025-09-02 15:56:47,投资理财,余额宝-转出到银行卡,余额宝-转出到银行卡,0,11.23,余额宝,交易成功
116,2025-08-28 13:14:11,账户存取,提现,提现-实时提现,1,805.98,中国银行储蓄卡(7633),交易成功


In [15]:
cleaned.payment.unique()

array(['中国银行储蓄卡(7633)', '余额', '账户余额', '余额宝', nan, '账户余额&碰一下立减'],
      dtype=object)

In [14]:
cleaned.loc[cleaned['payment'].isna()]

,time,type,counterparty,name,direction,amount,payment,status
62,2025-08-25 18:33:00,商业服务,杭州闲鱼信息技术有限公司,分账-基础软件服务费(4699411203857517145)扣款,0,4.2,NaN,交易成功
68,2025-08-23 21:17:24,收入,****6,任天堂Switch Lite黄色港版,1,700.0,NaN,交易成功
72,2025-08-22 14:02:07,收入,**伟,收钱码收款,1,200.0,NaN,交易成功


In [120]:
def cvt_record(source: pd.DataFrame, idx= False):
    source = source.copy()
    if idx:
        source = source.iloc[idx]
    record = pd.DataFrame(columns=['date', 'source', 'category', 'name', 'amt (RMB)', 'amt (Foreign)', 'balance', 'note'])
    record['date'] = source['time'].apply(lambda x: datetime.strftime(x, '%y%m%d'))
    record['source'] = source['payment']
    record['name'] = source['counterparty']
    record['amt (RMB)'] = source.apply(lambda x:
    -x['amount'] if x['direction'] == 0
    else x['amount'],
    axis=1)

    record = record.sort_values(by='date', ascending=True)
    
    return record

In [121]:
cvt_record(cleaned)

,date,source,category,name,amt (RMB),amt (Foreign),balance,note
114,250813,余额宝,NaN,余额宝-2025.08.12-收益发放,0.04,NaN,NaN,NaN
113,250813,余额宝,NaN,美团,-147.00,NaN,NaN,NaN
112,250813,余额宝,NaN,晓丰收柴火铁锅炖（大安店）,-24.00,NaN,NaN,NaN
111,250813,余额宝,NaN,美团,-256.50,NaN,NaN,NaN
117,250814,余额宝,NaN,余额宝-转入,2225.40,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4,250914,中国银行储蓄卡(7633),NaN,杭州杭港地铁五号线有限公司,-4.00,NaN,NaN,NaN
3,250915,中国银行储蓄卡(7633),NaN,饿了么,-16.70,NaN,NaN,NaN
2,250915,中国银行储蓄卡(7633),NaN,滴滴出行,-28.90,NaN,NaN,NaN
1,250916,中国银行储蓄卡(7633),NaN,绿城留庄(杭州)酒店管理有限公,-1.50,NaN,NaN,NaN
